HOW chatGPT interface works ?

1) Chat GPT is an app
2) It uses Open AI Apis to interact with llms
3) these llms could be GPT-3.5, GPT-4 turbo, etc.



But why people don't want to use the open ai api
They want their own LLM - Bcoz. of open AI cost incurred with using its API
Also, GPT don't has access to private data(of company/org.) and getting real time data from google and internet.

But still you can use "Hugging face Bloom" as an free open source model

Langchain - framework that allows you to build applications using LLMs.

In [3]:
%pip install -qU "langchain[cohere]"

Note: you may need to restart the kernel to use updated packages.


In [4]:
import getpass
import os

if not os.environ.get("COHERE_API_KEY"):
  os.environ["COHERE_API_KEY"] = getpass.getpass("Enter API key for Cohere: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("command-r-plus", model_provider="cohere")

model.invoke("Hello, world!")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={'id': 'eadea9f0-467f-4931-b5bd-4b1e9e9723d6', 'finish_reason': 'COMPLETE', 'content': 'Hello! How can I help you today?', 'token_count': {'input_tokens': 197.0, 'output_tokens': 9.0}}, response_metadata={'id': 'eadea9f0-467f-4931-b5bd-4b1e9e9723d6', 'finish_reason': 'COMPLETE', 'content': 'Hello! How can I help you today?', 'token_count': {'input_tokens': 197.0, 'output_tokens': 9.0}}, id='run-0124eb2e-333d-481a-a046-7c86a212e382-0', usage_metadata={'input_tokens': 197, 'output_tokens': 9, 'total_tokens': 206})

In [ ]:
# %pip install -qU "langchain[together]"

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import getpass
# import os

# if not os.environ.get("TOGETHER_API_KEY"):
#   os.environ["TOGETHER_API_KEY"] = getpass.getpass("Enter API key for Together AI: ")

# from langchain.chat_models import init_chat_model

# model2 = init_chat_model("mistralai/Mixtral-8x7B-Instruct-v0.1", model_provider="together")

# model2.invoke("Hello, world!")

AIMessage(content=" Hello, human! It's nice to see you're interested in starting a conversation with me. I'm here to help answer any questions you might have, so feel free to ask me about a wide range of topics, from general knowledge and facts to programming and technology. I can also assist you with tasks such as setting reminders, providing language translations, and offering real-time weather updates. What can I help you with today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 12, 'total_tokens': 104, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'system_fingerprint': None, 'id': 'nrLRhJt-4yUbBN-935331c98b2a59ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-ba6b1e31-b26e-4d60-ae95-10edd5ee1cb1-0', usage_metadata={'input_tokens': 12, 'output_tokens': 92, 'total_tokens': 104, 'input_token_details': {}, 'output_token_details': {}})

In [8]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Sugges a fency name for it."
)

prompt_template.format(cuisine="Italian")

'I want to open a restaurant for Italian food. Sugges a fency name for it.'

In [10]:
# concepts of chain in langchain
from langchain.chains import LLMChain

chain=LLMChain(llm=model, prompt=prompt_template)
chain.run("Mexican")

'Here are a few potential names for a fancy Mexican restaurant:\n\n- La Casa de los Sabores (The House of Flavors)\n- El Comedor Elegante (The Elegant Dining Room)\n- La Mesa de México (The Table of Mexico)\n- Los Sabores de la Vida (The Flavors of Life)\n- La Cocina de las Maravillas (The Wonder Kitchen)\n- El Paraíso Mexicano (The Mexican Paradise)\n- La Fiesta de los Sentidos (The Feast of the Senses)\n- Los Placeres de la Mesa (The Pleasures of the Table)\n- La Cantina de los Sueños (The Tavern of Dreams)\n- El Rincón de México (The Corner of Mexico)'

# Creating sequential chain
1) above we got name of restraunt as the first chain
2) now, we can have the second chain where u pass restraunt name as input 
and it will give food menu items to include in your restraunt

what are the sequential chain ?
Nothinh just the output of first thing will act as input of other thing

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name=PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Sugges a fency name for it(only one), just give the name in response."
)

# add a ouptut key here for the next chain to get it 
name_chain=LLMChain(llm=model, prompt=prompt_template_name, output_key='restaurant_name')

prompt_template_items=PromptTemplate(
    input_variables=['restaurant_name'],
    template=" Suggest some food menu items(names only) for restaraunt {restaurant_name} food. Return in a list format, just give the list items nothing else."
)

food_items_chain=LLMChain(llm=model, prompt=prompt_template_items, output_key='menu_items')

# --------------------------------------------------------------------------------
# Creating a sequential chain for above two chains

from langchain.chains import SimpleSequentialChain

chain=SimpleSequentialChain(chains = [name_chain, food_items_chain])

response=chain.run("Indian")

print(response)

Here is a list of suggested food menu items for an Indian restaurant with a fancy name like "Masala Mahal":

- Butter Chicken Supreme
- Dal Makhani Royale
- Palak Paneer Delight
- Malai Kofta Extravaganza
- Chicken Tikka Masala Fiesta
- Lamb Korma Celebration
- Shrimp Biryani Festival
- Vegetable Kebab Sensation
- Naan Bread Symphony
- Gulab Jamun Finale

 These menu item names are designed to be attention-grabbing and elegant, befitting the ambiance of a fancy Indian restaurant.


Above was a simple sequential chain -> notice the output of first chain(i.e. the Restaurant name is not outputted in the result, its hidden as of now)

To get both the output of first and second chain, and also provide multiple inputs to a chain, Use a different method called -> 'Sequential Chain' instead of 'simple Sequential chain'


In [ ]:
from langchain.chains import SequentialChain

seq_chain=SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables=['cuisine'],
    output_variables=['restaurant_name', 'menu_items'],
    verbose=True
)

# no run here to run a sequential chain
seq_chain({'cuisine':'Arabic'})



/tmp/ipykernel_133993/728699918.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  seq_chain({'cuisine':'Arabic'})




> Entering new SequentialChain chain...

> Finished chain.


{'cuisine': 'Arabic',
 'restaurant_name': 'Here is a suggestion for a fancy name for an Arabic food restaurant: "Al Sultan"',
 'menu_items': 'Here are some menu item suggestions for an Arabic food restaurant named "Al Sultan":\n\n- Appetizers:\n    - Sultan\'s Mezze Platter\n    - Hummus with Lamb\n    - Warak Enab\n    - Batata Harra\n- Salads:\n    - Fattoush Salad\n    - Tabbouleh\n    - Cucumber Yogurt Salad\n- Main Courses:\n    - Lamb Kabsa\n    - Chicken Shawarma\n    - Mixed Grill Platter\n    - Falafel Wrap\n- Desserts:\n    - Baklava Ice Cream\n    - Kunafa Cups\n    - Dates with Cream\n    - Arabic Coffee Mousse\n- Beverages:\n    - Mint Lemonade\n    - Arabian Nights Iced Tea\n    - Rose Milk\n    - Qahwa (Arabic Coffee)\n\n These suggestions offer a mix of traditional Arabic dishes and modern interpretations, providing a range of options for customers to choose from.'}